In [1]:
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba
import bw2io as bi
from tqdm import tqdm

In [2]:
bd.__version__, bc.__version__, ba.__version__, bi.__version__

((4, 0, 'DEV18'), (2, 0, 'DEV12'), (0, 11, 4), (0, 9, 'DEV9'))

In [3]:
bd.projects.set_current("eco-ex")

In [4]:
bd.databases

Databases dictionary with 4 object(s):
	biosphere3
	ecoinvent-3.8-cutoff
	exiobase-3.8.1-monetary
	exiobase-3.8.1-monetary biosphere

In [6]:
for act in tqdm(bd.Database("ecoinvent-3.8-cutoff")):
    for exc in act.technosphere():
        if exc.get('rescaled'):
            continue
        elif exc.input['database'] == 'exiobase-3.8.1-monetary':
            exc['amount'] = exc['amount'] / 1e6
            exc['rescaled'] = True
            exc.save()

100%|██████████| 19565/19565 [24:39<00:00, 13.22it/s] 


In [7]:
bd.Database("ecoinvent-3.8-cutoff").process()

Test to make sure we get hybridization included in our results:

In [8]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

In [9]:
elec = bd.get_activity(
    name='market group for electricity, low voltage',
    unit='kilowatt hour',
    location='US',
    database="ecoinvent-3.8-cutoff"
)

In [10]:
lca = bc.LCA({elec: 1}, ipcc)
lca.lci()
lca.lcia()

In [11]:
lca.score

0.5132577575434085

In [12]:
ca = ba.ContributionAnalysis()

In [13]:
ca.annotated_top_processes(lca)[:10]

[(0.06150752808337321,
  0.05768271575216754,
  'electricity production, hard coal' (kilowatt hour, US-RFC, None)),
 (0.060475898943734785,
  0.05366734989829976,
  'electricity production, hard coal' (kilowatt hour, US-SERC, None)),
 (0.0469929335612983,
  0.12446697728995236,
  'electricity production, natural gas, combined cycle power plant' (kilowatt hour, US-SERC, None)),
 (0.03208818683514746,
  0.026534124209094238,
  'electricity production, lignite' (kilowatt hour, US-MRO, None)),
 (0.027595092896085405,
  0.022176951553432578,
  'electricity production, lignite' (kilowatt hour, US-SERC, None)),
 (0.026368011368282703,
  0.022076589773994407,
  'electricity production, lignite' (kilowatt hour, US-WECC, None)),
 (0.023556029894962405,
  0.01972226124418963,
  'electricity production, lignite' (kilowatt hour, US-TRE, None)),
 (0.021918519805914614,
  0.06190924369444999,
  'electricity production, natural gas, combined cycle power plant' (kilowatt hour, US-RFC, None)),
 (0.01672

In [41]:
bi.backup_project_directory("eco-ex")

Creating project backup archive - this could take a few minutes...
